In [34]:
import pandas as pd
import numpy as np
import os, sys, glob, inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
import epri_mc_lib as mc
from importlib import reload
reload(mc)


<module 'epri_mc_lib' from '/home/marie-anne/code/Oct20_EPRI/Task1/NB/epri_mc_lib.py'>

## Generate synthetic data

This notebook generates synthetic data from sample measurements to give a measure of the uncertainty of the data. When drawing the new samples the features are assumed to be independent. A new sample for a given condition is created by drawing a value for each feature from a normal distribution with the mean and standard deviation of that feature for that condition. 1000 synthetic samples are created for each condition.

Output file is saved in Data/Merged_data/ALL_TUBE_PIPE_simulated.csv

In [35]:
data_path = "../../Data/Merged_data"
merged_data = pd.read_csv(os.path.join(data_path, 'ALL_TUBE_PIPE_merge_1.csv'), 
                 index_col=0)


In [36]:
num_samples_generated = 1000

For now we're throwing out the pipe data and only working with tubes.

In [37]:
tube, pipe, tube_wo_blind, tube_blind = mc.get_subsample_df(merged_data)

In [39]:
final_generated_data = pd.DataFrame(columns = ["Condition"] + mc.data_generation_values)

for index, row in tube.iterrows():
    generated_samples = pd.DataFrame(num_samples_generated * [index])
    generated_samples.columns = ["Condition"]
    for value_col, std_col in zip(mc.data_generation_values, mc.data_generation_stds):
        generated_samples[value_col] = pd.DataFrame(np.random.normal(tube.loc[index,value_col], 
              tube.loc[index,std_col], num_samples_generated))
    final_generated_data = final_generated_data.append(generated_samples, ignore_index = True)

print(final_generated_data)

      Condition  TEP_mean_uV_C  Absorption_avg_500  backscatter_avg  \
0          T_B1      12.887224            0.000252         0.020317   
1          T_B1      12.904167            0.000194         0.011580   
2          T_B1      12.848012            0.000247         0.027738   
3          T_B1      12.968064            0.000210         0.019859   
4          T_B1      12.927068            0.000233         0.013641   
...         ...            ...                 ...              ...   
15995   T_HAZ_T      12.948355            0.000329         0.020244   
15996   T_HAZ_T      12.940969            0.000244         0.022487   
15997   T_HAZ_T      13.038192            0.000599         0.029502   
15998   T_HAZ_T      13.044431            0.000271         0.020341   
15999   T_HAZ_T      12.895002            0.000371         0.020657   

       Absorption_avg_50             A         B         p  \
0               0.001931  1.202073e-07  0.001092  0.221759   
1               0.00112

In [40]:
final_generated_data.to_csv(os.path.join(data_path, 'ALL_TUBE_PIPE_simulated.csv'), index=False)